#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/billing")
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='Manage billing in your account - Docs by LangChainOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationSetupOverviewCreate an account and API keySet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesFAQsCloud architecture and scalabilityRegions FAQAuthentication methodsData purging for complianceRelease versionsOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationSetupManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSel

In [8]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [11]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='Manage billing in your account - Docs by LangChainOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationSetupOverviewCreate an account and API keySet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesFAQsCloud architecture and scalabilityRegions FAQAuthentication methodsData purging for complianceRelease versionsOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationSetupManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSel

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [14]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [15]:
vectorstoredb

In [16]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'\u200bOptimization 2: limit usage\nIn the previous section, you managed data retention settings to optimize existing spend. In this section, you will use usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics tracked on the usage graph. You can use these in tandem to have granular control over spend.\nTo set limits, navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the bottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along with a cost estimate:\n\nStart by setting limits on production usage, since that is where the majority of spend comes from.\n\u200bSet a good total traces limit\nPicking the right total traces limit depends on the expected load of traces that you will send to LangSmith. It is important to consider potential growth before setting a limit. For example:'

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [19]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001AC3CC04410>, default_metadata=(), model_kwargs={})
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [20]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'LangSmith has two usage limits: total traces and extended traces.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [21]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [22]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [23]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AC3B76A3C0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
  

In [24]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'LangSmith has two usage limits: total traces and extended retention traces.'

In [25]:

response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='64f93a17-8d1e-4903-90bc-64825c563de9', metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='\u200bOptimization 2: limit usage\nIn the previous section, you managed data retention settings to optimize existing spend. In this section, you will use usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics tracked on the usage graph. You can use these in tandem to have granular control over spend.\nTo set limits, navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the bottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along with a cost estimate:\n\nStart by setting limits on production usage, sin

In [26]:
response['context']

[Document(id='64f93a17-8d1e-4903-90bc-64825c563de9', metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='\u200bOptimization 2: limit usage\nIn the previous section, you managed data retention settings to optimize existing spend. In this section, you will use usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics tracked on the usage graph. You can use these in tandem to have granular control over spend.\nTo set limits, navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the bottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along with a cost estimate:\n\nStart by setting limits on production usage, since that is where the majority of spend comes from.\n\u200bSet a good total traces l